# HW3-2: MNIST 手写数字识别 （80分）

- 本次作业的主题是使用深度学习技术对`MNIST`手写数字数据集进行分类。
- 要求使用两种不同的深度学习模型，分别是多层感知机--`MLP`和卷积神经网络--`CNN`。
- 本次作业的目的是让同学们了解深度学习模型的构建和训练过程，以及对经典视觉任务处理效果的对比。

## 评分标准（更细致的评分标准见对应部分）

- 数据读取和预处理：5分
- 基于`MLP`的手写数字分类：30分 
- 基于`CNN`的手写数字分类：30分
- 结果讨论与对比分析：15分


---

## 第一部分：数据读取和预处理

你可以在Canvas上下载如下两个文件：
- 训练数据：`mnist_train.csv`
- 测试数据：`mnist_test.csv`

这两个文件包含了Mnist数据集的训练集和测试集。每一行的第一个数字表示标签，后面的784数字是28x28的图片像素值。
- 请你正确读取并分别展示训练集和测试集的第一个样本（图像+label）。

### 评分细则
- 数据读取：2分
- 数据展示：3分

In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('mnist_train.csv')
test_df = pd.read_csv('mnist_test.csv')

train_df.head()
test_df.head()
# Code Here

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---
## 第二部分：基于`MLP`的手写数字分类

- 请你构建一个多层感知机模型，在Mnist数据集上进行训练和测试。
- 可以使用`Pytorch`或者`Tensorflow`等深度学习框架。
- 请你展示模型的训练过程（Loss曲线）和测试结果（分类精度）。

### 评分细则
- 模型构建：10分
- 模型训练：10分 - 要求Loss曲线收敛
- 模型测试：10分 - 要求分类精度达到90%以上

In [ ]:
# Code here
import torch
import torch.nn as nn
import tqdm as tqdm
from sklearn.model_selection import train_test_split


class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # 28*28=784
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 classes for digits 0-9

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
# Convert DataFrame to PyTorch tensors
def df_to_tensor(df):
    labels   = torch.as_tensor(df.iloc[:, 0].values, dtype=torch.long)
    features = torch.as_tensor(df.iloc[:, 1:].values, dtype=torch.float32)
    return features, labels

model=SimpleNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

learning_rate = 0.001
batch_size = 64

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

train_features, train_labels = df_to_tensor(train_df)

train_features = train_features.to(device)
train_labels   = train_labels.to(device)
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

def train_model(model, train_loader,val_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct     = 0
        total       = 0
        
        for features, labels in tqdm.tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            optimizer.zero_grad()
            
            outputs = model(features)                   
            loss    = criterion(outputs, labels)        
            loss.backward()                            
            optimizer.step()                            

            batch_size     = labels.size(0)
            running_loss  += loss.item() * batch_size
            
            preds          = outputs.argmax(dim=1)
            correct      += (preds == labels).sum().item()
            total        += batch_size
        
        # 计算 epoch 平均损失和准确率
        epoch_loss = running_loss / total
        accuracy   = 100 * correct / total
        
        print(
            f'Epoch {epoch+1}/{epochs} — '
            f'Loss: {epoch_loss:.4f} — '
            f'Accuracy: {accuracy:.2f}%'
        )
        
        for features, labels in val_loader:
            model.eval()
            outputs = model(features)
            loss = criterion(outputs, labels)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        print(f'Validation Loss: {loss.item():.4f} — Validation Accuracy: {100 * correct / total:.2f}%')
            

test_features, test_labels = df_to_tensor(test_df)
test_features = test_features.to(device)
test_labels   = test_labels.to(device)        
test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'test Accuracy: {accuracy:.2f}%')

train_model(model, train_loader,val_loader, criterion, optimizer, epochs=10)
evaluate_model(model, test_loader)


Epoch 1/10: 100%|██████████| 750/750 [00:01<00:00, 692.59it/s]


Epoch 1/10 — Loss: 0.3506 — Accuracy: 90.99%
Validation Loss: 0.2932 — Validation Accuracy: 91.47%


Epoch 2/10: 100%|██████████| 750/750 [00:00<00:00, 835.63it/s]


Epoch 2/10 — Loss: 0.1576 — Accuracy: 95.36%
Validation Loss: 0.3595 — Validation Accuracy: 95.40%


Epoch 3/10: 100%|██████████| 750/750 [00:00<00:00, 799.34it/s]


...

Epoch 8/10 — Loss: 0.0707 — Accuracy: 97.89%
Validation Loss: 0.3973 — Validation Accuracy: 97.51%


Epoch 9/10: 100%|██████████| 750/750 [00:00<00:00, 769.97it/s]


Epoch 9/10 — Loss: 0.0621 — Accuracy: 98.13%
Validation Loss: 0.8199 — Validation Accuracy: 97.83%


Epoch 10/10: 100%|██████████| 750/750 [00:00<00:00, 764.53it/s]


Epoch 10/10 — Loss: 0.0674 — Accuracy: 98.07%
Validation Loss: 1.0631 — Validation Accuracy: 97.75%
Accuracy: 96.71%


In [11]:
import torch
import torch.nn as nn
import tqdm
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, DataLoader, Subset

# 假设 train_features/train_labels 都是 CPU 上的 Tensor
train_features, train_labels = df_to_tensor(train_df)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. 整体 Dataset
full_dataset = TensorDataset(train_features, train_labels)

# 2. 定义五折拆分器
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(
        skf.split(train_features.numpy(), train_labels.numpy()), 1):
    
    print(f"\n>>> Fold {fold}")
    # 3. 构造每折的 DataLoader
    train_subset = Subset(full_dataset, train_idx)
    val_subset   = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
    val_loader   = DataLoader(val_subset,   batch_size=64, shuffle=False)
    
    # 4. 每折都要重新初始化模型和优化器
    model = SimpleNN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    
    # 5. 训练若干 epoch
    for epoch in range(10):
        model.train()
        total_loss = total_correct = total_samples = 0
        for x, y in tqdm.tqdm(train_loader, desc=f"Fold{fold} Epoch{epoch+1}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            
            preds = logits.argmax(dim=1)
            total_correct += (preds == y).sum().item()
            total_loss   += loss.item() * y.size(0)
            total_samples+= y.size(0)
        
        print(f"  Train — loss: {total_loss/total_samples:.4f}, "
              f"acc: {100*total_correct/total_samples:.2f}%")
        
        # 验证
        model.eval()
        val_correct = val_samples = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                preds = logits.argmax(dim=1)
                val_correct += (preds == y).sum().item()
                val_samples += y.size(0)
        val_acc = 100 * val_correct / val_samples
        print(f"  Val   — acc: {val_acc:.2f}%")
    
    fold_metrics.append(val_acc)

# 6. 平均五折准确率
print(f"\nAverage 5-fold val accuracy: {sum(fold_metrics)/len(fold_metrics):.2f}%")



>>> Fold 1


Fold1 Epoch1: 100%|██████████| 750/750 [00:01<00:00, 515.80it/s]


  Train — loss: 0.3531, acc: 90.56%
  Val   — acc: 94.18%


Fold1 Epoch2: 100%|██████████| 750/750 [00:01<00:00, 475.26it/s]


  Train — loss: 0.1555, acc: 95.42%
  Val   — acc: 95.13%


...

  Train — loss: 0.0771, acc: 97.69%
  Val   — acc: 96.18%


Fold5 Epoch9: 100%|██████████| 750/750 [00:01<00:00, 463.89it/s]


  Train — loss: 0.0627, acc: 98.07%
  Val   — acc: 96.62%


Fold5 Epoch10: 100%|██████████| 750/750 [00:01<00:00, 469.33it/s]


  Train — loss: 0.0653, acc: 98.07%
  Val   — acc: 96.60%

Average 5-fold val accuracy: 96.63%


---
## 第三部分：基于`CNN`的手写数字分类

- 请你构建一个卷积神经网络模型，在Mnist数据集上进行训练和测试。
- 可以使用`Pytorch`或者`Tensorflow`等深度学习框架。
- 请你展示模型的训练过程（Loss曲线）和测试结果（分类精度）。

### 评分细则
- 模型构建：10分
- 模型训练：10分 - 要求Loss曲线收敛
- 模型测试：10分 - 要求分类精度达到95%以上

In [14]:
# Code here
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 7x7 after pooling
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2)  # 28x28 -> 14x14
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2)  # 14x14 -> 7x7
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

train_features, train_labels = df_to_tensor(train_df)
train_features = train_features.view(-1, 1, 28, 28).to(device)  # Reshape for CNN
train_labels = train_labels.to(device)
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_features, test_labels = df_to_tensor(test_df)
test_features = test_features.view(-1, 1, 28, 28).to(device)  # Reshape for CNN
test_labels = test_labels.to(device)
test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_model(model, train_loader, criterion, optimizer, epochs=10)
evaluate_model(model, test_loader)




Epoch 1/10: 100%|██████████| 750/750 [00:01<00:00, 680.26it/s]


Epoch 1/10 — Loss: 0.2624 — Accuracy: 94.88%
Validation Loss: 0.3528 — Validation Accuracy: 95.31%


Epoch 2/10: 100%|██████████| 750/750 [00:01<00:00, 735.44it/s]


Epoch 2/10 — Loss: 0.0559 — Accuracy: 98.26%
Validation Loss: 0.2526 — Validation Accuracy: 98.20%


Epoch 3/10: 100%|██████████| 750/750 [00:00<00:00, 753.43it/s]


...

Epoch 8/10 — Loss: 0.0208 — Accuracy: 99.35%
Validation Loss: 0.4847 — Validation Accuracy: 99.17%


Epoch 9/10: 100%|██████████| 750/750 [00:01<00:00, 648.02it/s]


Epoch 9/10 — Loss: 0.0177 — Accuracy: 99.47%
Validation Loss: 0.7306 — Validation Accuracy: 99.22%


Epoch 10/10: 100%|██████████| 750/750 [00:01<00:00, 622.62it/s]


Epoch 10/10 — Loss: 0.0197 — Accuracy: 99.42%
Validation Loss: 1.0428 — Validation Accuracy: 99.23%
Accuracy: 98.55%



---
## 第四部分：结果讨论与对比分析

- 请你对`MLP`和`CNN`两种模型的训练和测试结果进行对比分析。
- 你可以从模型的训练速度、模型的性能、模型的泛化能力等方面进行分析。
    - 训练速度：模型训练所需的时间
    - 模型性能：模型在测试集上的分类精度
    - 模型泛化能力：模型在未知数据上的表现，未知数据可以从Canvas上下载，在文件夹`unseen`中有5张手写数字图片。
- 请在下面的markdown cell中写下你的分析结果。

### 评分细则
- 结果讨论与对比分析：
    - 训练速度：4分
    - 模型性能：4分
    - 模型泛化能力：5分
    - 其他：2分